# Obtain data of AGN from SIMBAD and filtering the objects with data of Ne II, Ne III and Ne V in infrared from NED 

In this notebook we develop the code to Obtain the data of AGNs type Sy1 (Seyfert 1), Sy2 (Seyfert 2) and SyG (Seyfert Galaxies) from **[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** (SMB) and then search each object from the SMB sample in **[NED](https://ned.ipac.caltech.edu/)** to obtain the photometric data and filter all the objects from the sample that contain data of the emission lines of Ne II, Ne III and Ne V in infrared.

The part of the code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

In [ ]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [1]:
from datetime import date
import time

import pyvo
import numpy as np
from astroquery.ned import Ned

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [2]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [3]:
today = date.today()
today

datetime.date(2021, 10, 7)

We extract the object name (or in this case the main id) from the sample result.

In [22]:
simbad_sample_Sy = result.to_table()
obj_name_SMB=simbad_sample_Sy["main_id"]
simbad_sample_Sy

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes
,deg,deg,,,,km / s,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object
LEDA 165630,212.38023750000002,5.192030555555555,2009ApJS..182..543A,Sy2,2011A&A...528A.124C,69068.1659042559,0.2644,z,68,IR |QSO|Sy2|G |G
2MASS J10034653+1222009,150.9439585861228,12.3669990150726,2018yCat.1345....0G,Sy1,2012ApJS..203...21A,71464.57474536476,0.27514,z,17,IR |QSO|AGN|X |BiC|QSO|Sy1|Sy1|AGN
2XMMi J032518.4-060739,51.326875,-6.1273888888888886,2005AJ....129.1755A,Sy1,2009ApJS..182..543A,126754.92483174153,0.57005,z,13,QSO|X |Sy1|AGN
USNO-A2.0 0825-00787515,52.0941222224393,-6.7266691804094,2018yCat.1345....0G,Sy1,2009ApJS..182..543A,93552.23834660952,0.38102,z,15,QSO|Sy1|AGN|* |UV
2SLAQ J033852.90+001904.7,54.7204660377791,0.3179910253349,2018yCat.1345....0G,Sy1,2009ApJS..182..543A,108224.28742924411,0.45941,z,25,X |QSO|QSO|AGN|* |Sy1|AGN
2SLAQ J034007.62-004632.5,55.0317750936304,-0.7757204237907,2018yCat.1345....0G,Sy1,2009ApJS..182..543A,138971.58950556486,0.65175,z,19,QSO|QSO|Sy1|* |QSO|AGN
2XMM J032328.4-371009,50.86844,-37.16924,2012ApJ...756...27L,Sy1,,--,--,,1,X |Sy1
2MASS J13241017+6410056,201.042164402469,64.1681640803473,2018yCat.1345....0G,Sy1,2012ApJS..203...21A,89131.01929523506,0.35875,z,15,IR |QSO|AGN|X |QSO|* |Sy1|AGN|X


The sample results is saved in a .csv file:

In [5]:
simbad_sample_Sy.write(today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.csv',
                    format='ascii.csv',overwrite=True)

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is because for some objects when we search the information in [NED](https://ned.ipac.caltech.edu/) using their SIMBAD main id  it doesn't reconigze it, so we will use coordinates instead to get the NED object name

In [6]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively. This values are equivalents to the following frequencies which ara related to different labels of "Observed Passband" in NED database as we ilustrate in the next list:

* Frequency: 1.92e+13 Hz: [NeIII] 15.6 microns
* Frecuency: 1.93e+13 Hz: [Ne III] 15.56 (IRS), [NeIII] 15.6 Spitzer, [Ne III] (Spitzer), [Ne III] 15.5 (IRS)
* Frecuency: 2.09e+13 Hz: [Ne V] 14.32 (IRS), [Ne V] 14.32 Spitzer, [Ne V] 14.32, [Ne V] 14.3 Spitzer, [Ne V] 14.3 microns
* Frequency: 2.10e+13 Hz: [Ne V] 14.3 (IRS)
* Frequency: 2.34e+13 Hz: [Ne II] 12.81 (IRS), [Ne II] 12.8 Spitzer, [Ne II] 12.8 (IRS), [Ne II] 12.81, NEII (VLT), 12.81 microns (VLT) 

In the variable **Ne_IR_Freq** we define a list with the those frequency values

In [7]:
Ne_IR_Freq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0]

The next part of the code use **[Astroquery](https://astroquery.readthedocs.io/en/latest/index.html)** package to get the "Frequency" column from the photometry table for each object of the SIMBAD sample or to get the NED object name from SIMBAD object coordiantes extracted before with SkyCoord. 

So, we will try to get the **"Photometry"** table using directly the SIMBAD object name. If this isn't recognized in NED, then we try to get the NED object name from the object coordinates and after that we will try again to get the "Photometry" table but now using the NED object name.

The filter for the objects with Ne Line Emissions in Infrared will be looking in the column **"Frequency"** of the table the values of the list defined in Ne_IR_Freq.

In [13]:
Ne_IR_obj = [] #List of objects with Ne-IR data

outfile = open('Obj_with_Ne-IR.txt','w') #We create and open a file to write the list of objects names with Ne_IR data
headerstring = f'#Objects with Ne II, Ne III and Ne V emission lines in IR (Created on {today}):\n#' #Creating the header
outfile.write(headerstring) #Writing the header

start = time.time()
for ind in range(len(simbad_sample_Sy)):
    try:
        phot_t = Ned.get_table(obj_name_SMB[ind], table='photometry') #We get the Photometry Table using SIMBAD names
        Fr=phot_t['Frequency'] #We get the Frequency column from the table
        if Ne_IR_Freq[0] in Fr or Ne_IR_Freq[1] in Fr or Ne_IR_Freq[2] in Fr or Ne_IR_Freq[3] in Fr or Ne_IR_Freq[4] in Fr:
            print(f"Ind#:{ind}, Id(SIMBAD): {obj_name_SMB[ind]}")
            #Writing the object names
            outstring = f'\n{obj_name_SMB[ind]}'
            outfile.write(outstring) 
            Ne_IR_obj.append(obj_name_SMB[ind]) #Save the object name in the list Ne_IR_obj
    except:
        try:
            result_t = Ned.query_region(cat_smb_sample[ind],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample 
            obj_name_NED=result_t["Object Name"] 
            phot_t = Ned.get_table(obj_name_NED, table='photometry')
            Fr=phot_t['Frequency']
            if Ne_IR_Freq[0] in Fr or Ne_IR_Freq[1] in Fr or Ne_IR_Freq[2] in Fr or Ne_IR_Freq[3] in Fr or Ne_IR_Freq[4] in Fr:
                print(f"Ind#:{ind}, Id(NED): {obj_name_NED[0]}")
                #Writing the object names
                outstring = f'\n{obj_name_NED[0]}'
                outfile.write(outstring) 
                Ne_IR_obj.append(obj_name_NED[0]) #Save the object name in the list Ne_IR_obj
        except:
            pass
end = time.time()

#we close the file with objects names
outfile.close()

print('\nTotal objects with emission of NeII or NeIII or NeV lines found: ',len(Ne_IR_obj))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

Ind#:0, Id(SIMBAD): LEDA  165630
Ind#:23, Id(SIMBAD): ESO 362-18
Ind#:100, Id(NED): 3C 334
Ind#:182, Id(SIMBAD): ESO 323-77
Ind#:342, Id(SIMBAD): NGC  7479
Ind#:400, Id(SIMBAD): M  81
Ind#:408, Id(SIMBAD): NGC  1241
Ind#:409, Id(SIMBAD): NGC  3516
Ind#:429, Id(SIMBAD): IRAS 15206+3342
Ind#:450, Id(SIMBAD): LEDA 1487448
Ind#:611, Id(SIMBAD): Mrk  937
Ind#:624, Id(SIMBAD): ESO 297-18
Ind#:644, Id(SIMBAD): IC  450
Ind#:790, Id(SIMBAD): Mrk  622
Ind#:858, Id(NED): WISEA J035734.06-053719.6
Ind#:1051, Id(SIMBAD): 2MASS J21035875-0728025
Ind#:1111, Id(SIMBAD): IRAS 15480-0344
Ind#:1411, Id(SIMBAD): Mrk  471
Ind#:1585, Id(SIMBAD): 2MASS J00105486+0014510
Ind#:1588, Id(NED): WISEA J005439.78+155447.2
Ind#:1868, Id(SIMBAD): LEDA  166692
Ind#:2265, Id(SIMBAD): NGC  4992
Ind#:2623, Id(SIMBAD): 3C 295
Ind#:2851, Id(SIMBAD): LEDA    3237
Ind#:2852, Id(SIMBAD): Mrk  478
Ind#:2950, Id(SIMBAD): NGC  1125
Ind#:2967, Id(SIMBAD): Mrk  279
Ind#:2989, Id(SIMBAD): Mrk 1502
Ind#:2991, Id(SIMBAD): PB  3894
In

Ind#:12120, Id(SIMBAD): UGC  5101
Ind#:12121, Id(SIMBAD): NGC  7469
Ind#:12127, Id(SIMBAD): ESO 339-11
Ind#:12593, Id(SIMBAD): NGC  7314
Ind#:13540, Id(SIMBAD): 3C  22.0
Ind#:13635, Id(SIMBAD): LEDA  165618
Ind#:14483, Id(SIMBAD): IRAS 11119+3257
Ind#:15767, Id(SIMBAD): 2MASS J15011320+2329082
Ind#:16931, Id(SIMBAD): NGC  3367
Ind#:17348, Id(SIMBAD): NGC  4968
Ind#:17383, Id(SIMBAD): NGC   660
Ind#:17416, Id(SIMBAD): Z  41-20
Ind#:17849, Id(SIMBAD): M  58
Ind#:18256, Id(SIMBAD): NGC  6926
Ind#:18259, Id(SIMBAD): NGC   973
Ind#:18540, Id(SIMBAD): NGC  4138
Ind#:18545, Id(SIMBAD): NGC  5395
Ind#:18552, Id(SIMBAD): NGC  7603
Ind#:18553, Id(SIMBAD): NGC  7213
Ind#:18555, Id(SIMBAD): UGC  1395
Ind#:18595, Id(SIMBAD): NGC   863
Ind#:18631, Id(SIMBAD): 2MASS J08105865+7602424
Ind#:18640, Id(SIMBAD): ICRF J052109.8+163822
Ind#:18643, Id(NED): PG 0947+396
Ind#:18660, Id(SIMBAD): ESO 121-28
Ind#:18682, Id(SIMBAD): Ton  182
Ind#:18684, Id(SIMBAD): 2MASX J14510879+2709272
Ind#:18702, Id(SIMBAD): N

In [19]:
Ne_IR_obj

['LEDA  165630',
 'ESO 362-18',
 '3C 334',
 'ESO 323-77',
 'NGC  7479',
 'M  81',
 'NGC  1241',
 'NGC  3516',
 'IRAS 15206+3342',
 'LEDA 1487448',
 'Mrk  937',
 'ESO 297-18',
 'IC  450',
 'Mrk  622',
 'WISEA J035734.06-053719.6',
 '2MASS J21035875-0728025',
 'IRAS 15480-0344',
 'Mrk  471',
 '2MASS J00105486+0014510',
 'WISEA J005439.78+155447.2',
 'LEDA  166692',
 'NGC  4992',
 '3C 295',
 'LEDA    3237',
 'Mrk  478',
 'NGC  1125',
 'Mrk  279',
 'Mrk 1502',
 'PB  3894',
 'ICRF J054236.1+495107',
 '3C 098',
 'NGC  2992',
 'NGC  2110',
 'UGC  3351',
 'NGC   788',
 'NGC  4235',
 'NGC  6860',
 'Mrk  520',
 'MCG+04-48-002',
 'ICRF J145907.5+714019',
 '2E  2503',
 'MCG+00-29-023',
 'Z 218-7',
 'Mrk  463E',
 'LEDA   27453',
 'IRAS 07598+6508',
 'NGC  4151',
 'NGC  1365',
 '2XMM J101756.8+390526',
 '3C  84',
 'ESO 141-55',
 'MCG+10-17-021',
 'Mrk  509',
 'NGC  7591',
 'Mrk  493',
 'Mrk 1239',
 'Mrk  231',
 'NGC  3079',
 'NGC  4355',
 'IRAS 13342+3932',
 'WISEA J165939.77+183436.7',
 'Mrk  876',

Las celdas a continuación son para pruebas:

In [20]:
x=100

In [21]:
simbad_sample_Sy[x]

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes
,deg,deg,,,,km / s,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object
2E 3648,245.0909104026793,17.6066535268064,2018yCat.1345....0G,Sy1,2012ApJS..203...21A,124401.06048262629,0.55517,z,396,QSO|QSO|QSO|* |Rad|IR |QSO|Sy1|Rad|X |G |BH?


In [17]:
cat_smb_sample[x]

<SkyCoord (ICRS): (ra, dec) in deg
    (348.94488333, -59.05442778)>

In [18]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.27.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 21 Model 2 Stepping 0, AuthenticAMD
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.3.1

astropy: 4.3.1
numpy  : 1.20.3
pyvo   : 1.1

Watermark: 2.2.0

